In [8]:
import settings
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
train_data = pd.read_csv(os.path.join(settings.PROCESSED_DATA_DIR, "training_data_processed.csv"), index_col =0)
submission_data = pd.read_csv(os.path.join(settings.PROCESSED_DATA_DIR, "test_data_processed.csv"), index_col =0)

In [9]:
features = list(train_data.columns)
print features
features.remove('status_group')

['funder', 'gps_height', 'installer', 'longitude', 'latitude', 'basin', 'region_code', 'ward', 'population', 'public_meeting', 'scheme_management', 'permit', 'construction_year', 'extraction_type', 'management', 'payment_type', 'water_quality', 'quantity', 'source', 'waterpoint_type', 'area', 'year_recorded', 'day_of_year', 'status_group']


In [10]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
for feature in features:
    train_data[feature] = std_scaler.fit_transform(train_data[feature])

In [11]:
train_data.corr()

,funder,gps_height,installer,longitude,latitude,basin,region_code,ward,population,public_meeting,...,management,payment_type,water_quality,quantity,source,waterpoint_type,area,year_recorded,day_of_year,status_group
funder,1.000000,-0.013313,0.548626,-0.140417,0.030899,-0.132931,-0.017551,0.011372,-0.010199,-0.013125,...,0.051717,0.049092,-0.034308,0.044472,-0.085970,0.049501,0.014573,-0.005321,0.059569,-0.029716
gps_height,-0.013313,1.000000,0.007099,0.001374,-0.004638,-0.161747,-0.183530,0.043365,0.135000,0.037212,...,-0.037574,-0.097413,0.140433,-0.030788,0.120627,-0.236776,0.108191,0.318397,-0.286829,-0.096423
installer,0.548626,0.007099,1.000000,-0.086653,0.008000,-0.102407,0.017972,0.002709,0.011742,-0.032630,...,-0.017065,0.022219,0.015118,0.054215,-0.101540,0.021211,0.018330,0.007668,0.009076,-0.020911
longitude,-0.140417,0.001374,-0.086653,1.000000,-0.313689,0.527194,0.142766,0.067815,0.070440,0.022822,...,-0.128229,0.081454,-0.059278,0.003744,-0.048212,-0.137815,-0.008359,0.110649,-0.390644,-0.054992
latitude,0.030899,-0.004638,0.008000,-0.313689,1.000000,-0.227540,-0.236473,0.009555,-0.010231,0.012603,...,0.019082,0.142730,-0.017274,0.126326,0.023077,0.143210,0.009833,0.157729,0.330936,0.010286
basin,-0.132931,-0.161747,-0.102407,0.527194,-0.227540,1.000000,0.139903,0.043095,0.066909,0.022196,...,-0.061137,-0.029994,-0.075481,-0.026697,-0.060986,0.009888,-0.020311,-0.136501,-0.266259,0.013531
region_code,-0.017551,-0.183530,0.017972,0.142766,-0.236473,0.139903,1.000000,0.029521,0.094086,-0.015365,...,-0.011493,0.140840,-0.063526,-0.066815,-0.129165,0.184616,0.001934,0.165543,-0.089591,0.083584
ward,0.011372,0.043365,0.002709,0.067815,0.009555,0.043095,0.029521,1.000000,0.029550,-0.015341,...,-0.002283,-0.003269,-0.011248,0.001812,-0.009992,-0.002715,0.040243,0.079735,-0.032344,0.012028
population,-0.010199,0.135000,0.011742,0.070440,-0.010231,0.066909,0.094086,0.029550,1.000000,0.006577,...,-0.045171,0.034484,-0.031137,0.031485,-0.090523,0.029208,0.033749,0.131754,-0.141250,-0.015202
public_meeting,-0.013125,0.037212,-0.032630,0.022822,0.012603,0.022196,-0.015365,-0.015341,0.006577,1.000000,...,0.121558,-0.043847,0.037716,-0.061874,0.044536,-0.069933,-0.090149,-0.011143,-0.051730,-0.062137


In [12]:
train_data.head()

,funder,gps_height,installer,longitude,latitude,basin,region_code,ward,population,public_meeting,...,management,payment_type,water_quality,quantity,source,waterpoint_type,area,year_recorded,day_of_year,status_group
id,,,,,,,,,,,,,,,,,,,,,
69572,0.913728,1.041235,1.299952,-0.058535,-1.443921,-1.244266,-0.244331,0.618717,-0.150403,0.304989,...,-0.134814,-1.547892,0.272735,-0.405305,0.955412,-0.883628,-0.456612,-0.961305,-0.490181,0
8776,-0.693271,1.054220,-0.417670,-0.150958,1.301376,-0.031179,0.267400,0.873303,0.212283,0.304989,...,1.820820,-0.458620,0.272735,0.838865,-0.520276,-0.883628,-0.456612,1.124750,-0.578169,0
34310,-0.063628,0.025525,2.224584,0.915626,0.705276,0.373184,0.324259,0.953699,0.148654,0.304989,...,-0.134814,1.175287,0.272735,-0.405305,-2.979756,-0.322280,-0.456612,1.124750,-0.677156,0
67743,1.560600,-0.584768,1.882940,1.311649,-1.906515,1.181908,4.247528,0.864929,-0.258573,0.304989,...,-0.134814,-0.458620,0.272735,-1.649474,-1.504068,-0.322280,-0.456612,1.124750,-0.985115,1
19728,-1.479542,-0.964216,-1.165212,-1.528808,1.416086,-0.031179,0.153682,1.059218,-0.381589,0.304989,...,-3.068266,-0.458620,0.272735,2.083035,-0.520276,-0.883628,-0.456612,-0.961305,0.840642,0


In [13]:
from sklearn.cross_validation import train_test_split    
train_set, test_set, train_set_labels, test_set_labels = train_test_split(train_data[features], train_data['status_group'].values, test_size=0.2, random_state=1)

## k-Nearest Neighbors

I start with a kNN model to see if i could accurately predict the operation of pumps given their similarities to other pumps.

In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

n_neighbors = [1,5,10,20]
for k in n_neighbors :
    knn_clf = KNeighborsClassifier(n_neighbors = k)
    knn_clf.fit(train_set,train_set_labels)
    train_predictions = knn_clf.predict(train_set)
    test_predictions = knn_clf.predict(test_set)
    print accuracy_score(train_predictions,train_set_labels),accuracy_score(test_predictions,test_set_labels)

0.998505860814 0.743771043771
0.829878574886 0.760437710438
0.791977945664 0.752777777778
0.757928407584 0.728282828283


| K | Accuracy on Training Set | Accuracy on Test Set  |
| :-------------: |:-------------:| :-----:|
| 1      | 99.80% | 74.94% |
| 5      | 83.12%     |   76.42% |
| 10 | 79.33%      |    75.61% |
| 20 | 76.28%      |    79.93% |

The best model is with k=5, but the accuracy is still low, so i will try SVM next.

## Logistic Regression

In order to optimize the logistic regression classifier, I will play around the penalty parameter (l1 or l2) and the C parameter (Inverse of regularization strength).

In [15]:
from sklearn.linear_model import LogisticRegression
c_values = [0.001,0.01,0.1,0.3,1,3,10]
penalites = ['l1','l2']
for p in penalites:
    for c in c_values:
        lr_clf = LogisticRegression(penalty=p, C=c)
        lr_clf.fit(train_set,train_set_labels)
        train_predictions = lr_clf.predict(train_set)
        test_predictions = lr_clf.predict(test_set)
        print p, c, accuracy_score(train_predictions,train_set_labels),accuracy_score(test_predictions,test_set_labels)

l1 0.001 0.590458553421 0.582744107744
l1 0.01 0.59856057577 0.590235690236
l1 0.1 0.599507565395 0.590824915825
l1 0.3 0.599654874892 0.591077441077
l1 1 0.599675919106 0.591161616162
l1 3 0.599633830678 0.591245791246
l1 10 0.599633830678 0.591245791246
l2 0.001 0.598813106336 0.591750841751
l2 0.01 0.599549653823 0.59132996633
l2 0.1 0.599654874892 0.591161616162
l2 0.3 0.59959174225 0.591245791246
l2 1 0.599633830678 0.591245791246
l2 3 0.599654874892 0.59132996633
l2 10 0.599654874892 0.59132996633


## Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=100 , max_depth=25, min_samples_split = 2, min_samples_leaf = 2 )
rf_clf.fit(train_set,train_set_labels)
train_predictions = rf_clf.predict(train_set)
test_predictions = rf_clf.predict(test_set)
print accuracy_score(train_predictions,train_set_labels),accuracy_score(test_predictions,test_set_labels)

0.926829268293 0.811868686869
